In [3]:
#importing the libraries which will be used
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer      #this is used for counting
from sklearn.metrics.pairwise import cosine_similarity           # this is to calculate cosine similarity

In [4]:
#these are the helping functions


def get_title_from_index(index):
	return df[df.index == index]["title"].values[0]

def get_index_from_title(title):
	return df[df.title == title]["index"].values[0]


In [5]:
#Here we are reading the datset

df = pd.read_csv("movie_dataset.csv")

In [6]:
#to display head

df.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


In [7]:
df.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [8]:
# here I am selecting features which I need for content based recommendation system


features = ['keywords','cast','genres','director']

In [9]:
# Create a column in DF which combines all selected features

# Here we are filling all the NaN values with empty string
for feature in features:
	df[feature] = df[feature].fillna('')

def combine_features(row):
	try:
		return row['keywords'] +" "+row['cast']+" "+row["genres"]+" "+row["director"]
	except:
		print ("Error:", row)        #If there is some errror then this message will be displayed

In [10]:
#axis = 1 or ‘columns’: apply function to each row.
df["combined_features"] = df.apply(combine_features,axis=1)   #Pandas.apply allow the users to pass a function and apply it 
                                                               #on every single value of the Pandas series.
                                                        

In [11]:
df.apply(combine_features,axis=1)

0       culture clash future space war space colony so...
1       ocean drug abuse exotic island east india trad...
2       spy based on novel secret agent sequel mi6 Dan...
3       dc comics crime fighter terrorist secret ident...
4       based on novel mars medallion space travel pri...
5       dual identity amnesia sandstorm love of one's ...
6       hostage magic horse fairy tale musical Zachary...
7       marvel comic sequel superhero based on comic b...
8       witch magic broom school of witchcraft wizardr...
9       dc comics vigilante superhero based on comic b...
10      saving the world dc comics invulnerability seq...
11      killing undercover secret agent british secret...
12      witch fortune teller bondage exotic island mon...
13      texas horse survivor texas ranger partner John...
14      saving the world dc comics superhero based on ...
15      based on novel fictional place brother sister ...
16      new york shield marvel comic superhero based o...
17      sea ca

In [12]:
df["combined_features"].head(10)


0    culture clash future space war space colony so...
1    ocean drug abuse exotic island east india trad...
2    spy based on novel secret agent sequel mi6 Dan...
3    dc comics crime fighter terrorist secret ident...
4    based on novel mars medallion space travel pri...
5    dual identity amnesia sandstorm love of one's ...
6    hostage magic horse fairy tale musical Zachary...
7    marvel comic sequel superhero based on comic b...
8    witch magic broom school of witchcraft wizardr...
9    dc comics vigilante superhero based on comic b...
Name: combined_features, dtype: object

In [13]:

#here I am  Creating count matrix from this new combined column
cv = CountVectorizer()

count_matrix = cv.fit_transform(df["combined_features"])

In [14]:
# Compute the Cosine Similarity based on the count_matrix
cosine_sim = cosine_similarity(count_matrix) 
movie_user_likes = "Avatar"

In [15]:
# Get index of this movie from its title
movie_index = get_index_from_title(movie_user_likes)


In [16]:
# here we are creating a list of tuples having similarity score as well as index
similar_movies =  list(enumerate(cosine_sim[movie_index]))

In [20]:
similar_movies[0:5]

[(0, 1.0000000000000004),
 (1, 0.10540925533894599),
 (2, 0.12038585308576921),
 (3, 0.03774256780481986),
 (4, 0.23094010767585033)]

In [18]:
# now we wil sort these list of tuples based on similarity score in decending order
#sorted is inbuilt function to sort
#lamda is used to sort it based on second index i.e cosine similarity
#reverse= true means in decending order
sorted_similar_movies = sorted(similar_movies,key=lambda x:x[1],reverse=True)

In [19]:
## now I will print the top 50 movies on the basis of cosine similarity
j=0
i=0
for element in sorted_similar_movies:
		print (get_title_from_index(element[0]))
		i=i+1
		if i>50:
			break

Avatar
Guardians of the Galaxy
Aliens
Star Wars: Clone Wars: Volume 1
Star Trek Into Darkness
Star Trek Beyond
Alien
Lockout
Jason X
The Helix... Loaded
Moonraker
Planet of the Apes
Galaxy Quest
Gravity
Alien³
Jupiter Ascending
The Wolverine
Silent Running
Zathura: A Space Adventure
Trekkies
Cargo
Wing Commander
Star Trek
Lost in Space
Babylon A.D.
The Fifth Element
Oblivion
Titan A.E.
AVP: Alien vs. Predator
The Empire Strikes Back
Dragonball Evolution
Superman Returns
Divergent
John Carter
The Black Hole
The Ice Pirates
Memoirs of an Invisible Man
Starship Troopers
The Astronaut's Wife
Machete Kills
Soldier
The Abyss
Damnation Alley
Men in Black
Space Cowboys
Space Dogs
The Time Machine
Sheena
Captain America: Civil War
Star Trek: Insurrection
Oz: The Great and Powerful
